## 6. Blocking vs. non‑blocking sockets

By default, socket operations **block**: `recv()` halts the thread until data arrives.  Setting `sock.setblocking(False)` makes calls raise `BlockingIOError` instead of waiting—useful for event loops (`select`, `asyncio`).

Two design patterns:
* **Select loop** – wait for readiness, then `recv()`.
* **Timeouts** – keep blocking sockets but call `settimeout(seconds)` to avoid hangs.

```python
import socket, select
s = socket.socket(); s.setblocking(False)
try:
    s.connect(('example.com', 80))
except BlockingIOError:
    pass  # expected for non‑blocking connect
ready,_,_ = select.select([], [s], [], 2)
if ready:
    s.send(b'')
```

### Quick check

1. `settimeout(0)` is equivalent to:
  a. blocking   b. non‑blocking

2. True / False Non‑blocking `recv()` returns empty bytes if no data.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – raises BlockingIOError.

</details>

## 7. Nagle’s algorithm & `TCP_NODELAY`

TCP coalesces small packets (Nagle) to reduce overhead; great for telnet, bad for latency‑sensitive realtime apps.  Disable with `setsockopt(IPPROTO_TCP, TCP_NODELAY, 1)` when you need each write to hit the wire immediately.

```python
import socket
s=socket.create_connection(('example.com',80))
s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
```

### Quick check

1. Disabling Nagle **increases**:
  a. bandwidth efficiency  b. per‑packet latency

2. True / False `TCP_NODELAY` affects UDP sockets.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – only TCP.

</details>

## 8. Port numbers & privileges

Ports 0‑1023 = *well‑known* (HTTP 80, HTTPS 443). On Unix, binding <1024 requires root.  Ephemeral ports (49152‑65535) chosen by OS for client sockets.  Avoid hard‑coding server ports already reserved (see `/etc/services`).

```bash
grep 8080 /etc/services | head -1  # list known mapping if any
```

### Quick check

1. Binding port 80 as normal user likely raises:
  a. PermissionError  b. Timeout

2. True / False Two sockets can bind same port/IP simultaneously with TCP.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – unless using SO_REUSEPORT semantics.

</details>

## 9. Byte order & the `struct` module

Network byte order is **big‑endian**. Convert integers with `struct.pack('!I', num)` or `socket.htonl()`. Mismatched endian renders data garbage across architectures.

```python
import struct
x=0x12345678
net=struct.pack('!I', x)
host=struct.unpack('!I', net)[0]
print(hex(host))
```

### Quick check

1. `'!H'` in struct format means:
  a. host endian short  b. network endian short

2. True / False Endian issues disappear if you send text JSON.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – JSON is ASCII chars.

</details>

## 10. DNS look‑ups and blocking `connect()`

`socket.getaddrinfo()` resolves names; it can stall seconds.  Use IP literals or run resolution in thread pool for low‑latency apps.  `connect()` includes DNS + TCP handshake + TLS by default.

```python
import socket, time
t=time.perf_counter(); socket.getaddrinfo('python.org', 443); print('lookup',time.perf_counter()-t)
```

### Quick check

1. DNS uses which protocol by default?
  a. UDP 53  b. TCP 80

2. True / False `getaddrinfo()` can return multiple records for IPv4/IPv6.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>